# Dados - Q3

In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import xarray

# Buscando os dados no CPTEC

In [2]:
import urllib
url = 'http://ftp.cptec.inpe.br/modelos/tempo/MERGE/GPM/DAILY/2021/09/MERGE_CPTEC_20210928.grib2'
temp_grib = "/home/github/TG/tmp/tmp2.grib2"
urllib.request.urlretrieve(url, temp_grib)


('/home/github/TG/tmp/tmp2.grib2', <http.client.HTTPMessage at 0x7f7ad2c73d00>)

## Lendo o arquivo GRIB2 e transformando pra netCDF

In [3]:
# OBS.: foi necessário instalar a engine 'pynio'. 
# < conda install -c conda-forge pynio >
ds = xarray.load_dataset(temp_grib, engine='pynio')
ds.to_netcdf('MERGE_CPTEC_20210928.nc')

In [29]:
ds.PREC_P0_L1_GLL0['lon_0']

<xarray.DataArray 'lon_0' (lon_0: 1001)>
array([239.95, 240.05, 240.15, ..., 339.75, 339.85, 339.95], dtype=float32)
Coordinates:
  * lon_0    (lon_0) float32 239.9 240.1 240.1 240.2 ... 339.7 339.8 339.9 340.0
Attributes:
    long_name:  longitude
    grid_type:  Latitude/Longitude
    units:      degrees_east
    Dj:         [0.1]
    Di:         [0.10000001]
    Lo2:        [339.95]
    La2:        [32.25]
    Lo1:        [239.95]
    La1:        [-60.05]

## Recortes

In [25]:
def longitude_formatter(val=float):
    '''
    Adapta longitude de 0 > 360 para -180 > 180
    
    Valores entre 0 e 180 são graus a Leste, estes se mantém.
    Enquanto se forem entre 0 e 180, subtrai-se 360.
    '''
    if val > 180:
        val -= 360
    return val

In [9]:
import geopandas as gpd

In [37]:
data = ds.PREC_P0_L1_GLL0.to_dataframe().reset_index()
data['lon_0'] = data['lon_0'].apply(lambda x: longitude_formatter(x))

In [38]:
gdf = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data['lon_0'],data['lat_0']))
gdf

,lat_0,lon_0,PREC_P0_L1_GLL0,geometry
0,-60.049999,-120.050003,0.00,POINT (-120.05000 -60.05000)
1,-60.049999,-119.949997,4.50,POINT (-119.95000 -60.05000)
2,-60.049999,-119.850006,5.25,POINT (-119.85001 -60.05000)
3,-60.049999,-119.750000,6.00,POINT (-119.75000 -60.05000)
4,-60.049999,-119.649994,6.00,POINT (-119.64999 -60.05000)
...,...,...,...,...
924919,32.250000,-20.449982,0.00,POINT (-20.44998 32.25000)
924920,32.250000,-20.349976,0.00,POINT (-20.34998 32.25000)
924921,32.250000,-20.250000,0.00,POINT (-20.25000 32.25000)
924922,32.250000,-20.149994,0.00,POINT (-20.14999 32.25000)


In [39]:
mascara = gpd.read_file('tiete.shp')

In [40]:
gdf.set_crs(mascara.crs)

,lat_0,lon_0,PREC_P0_L1_GLL0,geometry
0,-60.049999,-120.050003,0.00,POINT (-120.05000 -60.05000)
1,-60.049999,-119.949997,4.50,POINT (-119.95000 -60.05000)
2,-60.049999,-119.850006,5.25,POINT (-119.85001 -60.05000)
3,-60.049999,-119.750000,6.00,POINT (-119.75000 -60.05000)
4,-60.049999,-119.649994,6.00,POINT (-119.64999 -60.05000)
...,...,...,...,...
924919,32.250000,-20.449982,0.00,POINT (-20.44998 32.25000)
924920,32.250000,-20.349976,0.00,POINT (-20.34998 32.25000)
924921,32.250000,-20.250000,0.00,POINT (-20.25000 32.25000)
924922,32.250000,-20.149994,0.00,POINT (-20.14999 32.25000)


In [46]:
mascara.crs

<Geographic 2D CRS: EPSG:4674>
Name: SIRGAS 2000
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Latin America - SIRGAS 2000 by country
- bounds: (-122.19, -59.87, -25.28, 32.72)
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [48]:
gdf.set_crs(4674)
gdf.crs

In [50]:
tiete_clip = gpd.clip(gdf = gdf, mask = mascara['geometry'])

<ipython-input-50-83fd259c97fe>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4674

  tiete_clip = gpd.clip(gdf = gdf, mask = mascara['geometry'])


In [54]:
xarray.Dataset(tiete_clip.set_index(['lat_0','lon_0']))

<xarray.Dataset>
Dimensions:          (dim_0: 544)
Coordinates:
  * dim_0            (dim_0) MultiIndex
  - lat_0            (dim_0) float64 -23.85 -23.85 -23.85 ... -20.75 -20.75
  - lon_0            (dim_0) float64 -46.85 -46.75 -46.65 ... -49.95 -49.85
Data variables:
    PREC_P0_L1_GLL0  (dim_0) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    geometry         (dim_0) object POINT (-46.85000610351562 -23.85000038146...

In [20]:
def clip(data, shp):
    #precip = CPC.resample(time='m').sum()
    
    data = data.PREC_P0_L1_GLL0.to_dataframe().reset_index()
    
    gdf = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data['lon_0'],data['lat_0']))
    gdf = gdf.set_crs(4326)
    
    #gdf['PREC_P0_L1_GLL0'] = gdf['PREC_P0_L1_GLL0'].apply(lambda x: int(x))
    clipped = gpd.clip(gdf, shp['geometry'])
    clipped = clipped.groupby('time').mean().dropna()
    #clipped.to_pickle('/home/github/Climatology/Mod/Files/'+str(shp['Usina'])+'.pkl')
    return clipped

In [25]:
clip(ds, 'tiete.shx')

TypeError: string indices must be integers